# Working with Vector Tiles and PMTiles

## Introduction

## Learning Objectives

## Sample Datasets

## Installation and Setup

## Visualizing Vector Tiles Directly from Files

### Configuring JupyterHub Access (Optional)

### Visualizing Data from Vector Files

### Visualizing Data from Existing DuckDB Databases

## Converting Vector Data to PMTiles

### Understanding Coordinate Transformations for PMTiles

### Exporting a Global Cities Dataset to PMTiles

## Visualizing PMTiles

### Visualizing Local PMTiles with Martin Tile Server

### Visualizing Cloud-Hosted PMTiles

### Creating 3D Building Visualizations

### Simplified 3D Building Visualization

### Visualizing 2D Building Footprints

### Visualizing Overture Transportation Networks

### Visualizing Overture Places

## Key Takeaways

## Exercises

### Exercise 1: Visualizing Local Vector Files with On-the-Fly Tiling

### Exercise 2: Visualizing Data from DuckDB Database

### Exercise 3: Converting Spatial Data to PMTiles

### Exercise 4: Serving and Visualizing Local PMTiles

### Exercise 5: Exporting NYC Data to PMTiles

### Exercise 6: Multi-Layer PMTiles Visualization

### Exercise 7: Visualizing Cloud-Hosted Overture Data

### Exercise 8: Custom 3D Building Visualization

### Exercise 9: Overture Transportation Network Analysis

### Exercise 10: Complete Spatial Analysis to PMTiles Workflow